In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import glob

In [2]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from scipy import signal

In [24]:
chase_param = glob.glob('./../dataFolders/MuscaChasingMusca/Outputs\\*ChaseError*.csv')
chase_param

['./../dataFolders/MuscaChasingMusca/Outputs\\test_spiral_COM_ChaseErrorAnglesSpeedAcc.csv']

In [25]:
# speed_param = glob.glob('./../dataFolders/MuscaChasingMusca/Outputs/Sampled_1k/Speeds/*COM*.csv')
speed_param = glob.glob('./../dataFolders/MuscaChasingMusca/Outputs/*COM*_Fly*Speed*.csv')
speed_param

['./../dataFolders/MuscaChasingMusca/Outputs\\test_spiral_COM_FlyAnglularSpeedAcc.csv',
 './../dataFolders/MuscaChasingMusca/Outputs\\test_spiral_COM_FlyLinearSpeedAcc.csv']

In [26]:
raw_positions = glob.glob('./../dataFolders/MuscaChasingMusca/Outputs/*COM.csv')
raw_positions

['./../dataFolders/MuscaChasingMusca/Outputs\\test_spiral_COM.csv']

In [27]:
for chase in chase_param:
    chase_df = pd.read_csv(chase)
#     name = chase.split('\\')[1][:-53]
    name = chase.split('\\')[1].split('_')[0]
    
    speed_list = [speed for speed in speed_param if name  in speed]
    angular_speed_df = pd.read_csv([s for s in speed_list if "Anglular" in s][0])
    linear_speed_df = pd.read_csv([s for s in speed_list if 'Linear' in s][0])
    
    com_df = pd.read_csv([com for com in raw_positions if name in com][0])
    
    framerate = 1000 # sampled at 1k
    time_Axis = np.arange(len(chase_df))* 1e3/framerate # in milliseconds

In [28]:
print(chase)
print(name)

./../dataFolders/MuscaChasingMusca/Outputs\test_spiral_COM_ChaseErrorAnglesSpeedAcc.csv
test


In [29]:
# titles = titles = [
#     "", "", "θ (angle between chaser and chasee)",
#     "", "", "ω (turning angle of chaser)",
#     "", "", "ω̂ (rate of turning angle of chaser)"
# ]


fig = make_subplots(
    rows=3, cols=3,
#     subplot_titles=titles,
    specs=[
        [{"type": "scene", "rowspan": 3, "colspan": 2}, None, {"type": "xy"}],
        [None,                        None,              {"type": "xy"}],
        [None,                        None,              {"type": "xy"}]
          ])


################### the 3D chase plot #######################################################################
# 3D line plot to recreate the chase
line_trace_1 = go.Scatter3d(
    x = com_df.chaser_com_x, y = com_df.chaser_com_y, z = com_df.chaser_com_z,
    mode='lines',
    line=dict(color='black'),
    name='Line'
)

line_trace_2 = go.Scatter3d(
    x = com_df.chasee_com_x, y = com_df.chasee_com_y, z = com_df.chasee_com_z,
    mode='lines',
    line=dict(color='black', width=1),
    name='Line'
)

N = len(com_df)
# Markers every 10th point
marker_indices = np.arange(0, N, 10)

marker_trace_1 = go.Scatter3d(
    x=com_df.chaser_com_x[marker_indices],
    y=com_df.chaser_com_y[marker_indices],
    z=com_df.chaser_com_z[marker_indices],
    mode='markers',
    marker=dict(
        size=3,
        color='black',
        symbol='circle'
#         ['circle' if i % 2 == 0 else 'circle-open' for i in range(len(marker_indices))],
#         line=dict(width=1, color='black')
    ),
    name='Markers_chaser'
)

marker_trace_2 = go.Scatter3d(
    x=com_df.chasee_com_x[marker_indices],
    y=com_df.chasee_com_y[marker_indices],
    z=com_df.chasee_com_z[marker_indices],
    mode='markers',
    marker=dict(
        size=3,
        color='black',
        symbol='circle-open'
#         ['circle' if i % 2 == 0 else 'circle-open' for i in range(len(marker_indices))],
#         line=dict(width=1, color='black')
    ),
    name='Markers_chasee'
)


# Text labels every 50th point
text_indices = np.arange(0, N, 50)

text_trace_1 = go.Scatter3d(
    x=com_df.chaser_com_x[text_indices],
    y=com_df.chaser_com_y[text_indices],
    z=com_df.chaser_com_z[text_indices],
    mode='text',
    text=[str(i) for i in text_indices],
    textposition='top center',
    textfont=dict(size=10, color='black'),
    name='Labels_chaser'
)

text_trace_2 = go.Scatter3d(
    x=com_df.chasee_com_x[text_indices],
    y=com_df.chasee_com_y[text_indices],
    z=com_df.chasee_com_z[text_indices],
    mode='text',
    text=[str(i) for i in text_indices],
    textposition='bottom left',
    textfont=dict(size=10, color='black'),
    name='Labels_chasee'
)

# Create the figure

# Add all traces to the subplot
fig.add_trace(line_trace_1, row=1, col=1)
fig.add_trace(line_trace_2, row=1, col=1)
fig.add_trace(marker_trace_1, row=1, col=1)
fig.add_trace(marker_trace_2, row=1, col=1)
fig.add_trace(text_trace_1, row=1, col=1)
fig.add_trace(text_trace_2, row=1, col=1)

# Set the camera to view along the z-axis (xy plane)
fig.update_layout(
    scene_camera=dict(
        eye=dict(x=0., y=0., z=2.5)  # Adjust z to control the distance
    ),
    scene=dict(
        aspectmode='cube'  # Keeps the aspect ratio uniform
    )
)

############## add all the chase variables as line plots in second column ###############################
###### angle between the chasee and chaser ##############################################################
marker_trace_theta = go.Scatter(
    x=time_Axis[marker_indices],
    y=chase_df.loc[:,'chase_theta'][marker_indices],
    mode='markers',
    marker=dict(
        size=5,
        color='black',
        symbol='circle'
    ),
    name='theta'
)

fig.add_trace(go.Scatter(
    x = time_Axis, y = chase_df.loc[:,'chase_theta'],
    mode = 'lines'),
              row = 1, col = 3)
fig.add_trace(marker_trace_theta, row = 1, col = 3)


####### turning angle for the chaser ####################################################################
omega = angular_speed_df.loc[:,'chaser_theta']
omega_smoothed = signal.savgol_filter(omega, 10, 3)


fig.add_trace(go.Scatter(
        x = time_Axis, y = omega_smoothed
        , mode = 'lines'
        ),
             row = 2, col = 3)

######## rate of turning angle for the chaser ############################################################
omega_hat = np.diff(omega_smoothed)

marker_indices = np.arange(0, len(omega_hat), 20)
marker_trace_omega_hat = go.Scatter(
    x=time_Axis[marker_indices], 
    y=omega_hat[marker_indices],
    mode='markers',
    marker=dict(
        size=5,
        color='black',
        symbol='circle'
    ),
    name='omega_hat'
)

fig.add_trace(go.Scatter(
    x = time_Axis, y = omega_hat,
    mode = 'lines'),
              row = 3, col = 3)
fig.add_trace(marker_trace_omega_hat, row = 3, col = 3)

# Add manual annotations for column 3 titles
titles = [
    "θ (angle between chaser and chasee)",
    "ω (turning angle of chaser)",
    "ω̂ (rate of turning angle of chaser)"
]

for i, title in enumerate(titles):
    fig.add_annotation(
        text=title,
        xref="paper",
        yref="paper",
        x=0.85,  # far right (col 3)
        y=1.0 - (i * 1 / 2.8),  # top, middle, bottom
        xanchor="center",
        yanchor="bottom",
        showarrow=False,
        font=dict(size=12, color="black")
    )

# Final layout tweaks
fig.update_layout(
    height=800,
    width=1000,
#     title_text="Custom Titles for Column 3 Subplots",
    showlegend=False
)

fig.update_layout({
    'xaxis3': dict(title='Time (miliseconds)'),
    "yaxis3": dict(title="ω̂ (degree/s² * 1e3)", showgrid=True)
    })


fig.show()
# fig.write_html("../dataFolders/MuscaChasingMusca/Sampled1K_Figures/Chase_relationships/" + name + "theta_visualization.html")
fig.write_html("../dataFolders/MuscaChasingMusca/Outputs/" + name + "theta_visualization.html")

In [30]:
cross_Cor = np.correlate(omega_hat, chase_df.loc[:,'chase_theta'], mode='full')
x_Axis = np.arange(-len(omega_hat), len(omega_hat), 1)
fig = go.Figure(data=go.Scatter(x = x_Axis, y=cross_Cor))
fig.show()
fig.write_html("../dataFolders/MuscaChasingMusca/Sampled1K_Figures/Chase_relationships/" + name + "theta_cross_cor.html")

In [31]:
# titles = titles = [
#     "", "", "θ (angle between chaser and chasee)",
#     "", "", "ω (turning angle of chaser)",
#     "", "", "ω̂ (rate of turning angle of chaser)"
# ]


fig = make_subplots(
    rows=3, cols=3,
#     subplot_titles=titles,
    specs=[
        [{"type": "scene", "rowspan": 3, "colspan": 2}, None, {"type": "xy"}],
        [None,                        None,              {"type": "xy"}],
        [None,                        None,              {"type": "xy"}]
          ])


################### the 3D chase plot #######################################################################
# 3D line plot to recreate the chase
line_trace_1 = go.Scatter3d(
    x = com_df.chaser_com_x, y = com_df.chaser_com_y, z = com_df.chaser_com_z,
    mode='lines',
    line=dict(color='black'),
    name='Line'
)

line_trace_2 = go.Scatter3d(
    x = com_df.chasee_com_x, y = com_df.chasee_com_y, z = com_df.chasee_com_z,
    mode='lines',
    line=dict(color='black', width=1),
    name='Line'
)

N = len(com_df)
# Markers every 10th point
marker_indices = np.arange(0, N, 10)

marker_trace_1 = go.Scatter3d(
    x=com_df.chaser_com_x[marker_indices],
    y=com_df.chaser_com_y[marker_indices],
    z=com_df.chaser_com_z[marker_indices],
    mode='markers',
    marker=dict(
        size=3,
        color='black',
        symbol='circle'
#         ['circle' if i % 2 == 0 else 'circle-open' for i in range(len(marker_indices))],
#         line=dict(width=1, color='black')
    ),
    name='Markers_chaser'
)

marker_trace_2 = go.Scatter3d(
    x=com_df.chasee_com_x[marker_indices],
    y=com_df.chasee_com_y[marker_indices],
    z=com_df.chasee_com_z[marker_indices],
    mode='markers',
    marker=dict(
        size=3,
        color='black',
        symbol='circle-open'
#         ['circle' if i % 2 == 0 else 'circle-open' for i in range(len(marker_indices))],
#         line=dict(width=1, color='black')
    ),
    name='Markers_chasee'
)


# Text labels every 50th point
text_indices = np.arange(0, N, 50)

text_trace_1 = go.Scatter3d(
    x=com_df.chaser_com_x[text_indices],
    y=com_df.chaser_com_y[text_indices],
    z=com_df.chaser_com_z[text_indices],
    mode='text',
    text=[str(i) for i in text_indices],
    textposition='top center',
    textfont=dict(size=10, color='black'),
    name='Labels_chaser'
)

text_trace_2 = go.Scatter3d(
    x=com_df.chasee_com_x[text_indices],
    y=com_df.chasee_com_y[text_indices],
    z=com_df.chasee_com_z[text_indices],
    mode='text',
    text=[str(i) for i in text_indices],
    textposition='bottom left',
    textfont=dict(size=10, color='black'),
    name='Labels_chasee'
)

# Create the figure

# Add all traces to the subplot
fig.add_trace(line_trace_1, row=1, col=1)
fig.add_trace(line_trace_2, row=1, col=1)
fig.add_trace(marker_trace_1, row=1, col=1)
fig.add_trace(marker_trace_2, row=1, col=1)
fig.add_trace(text_trace_1, row=1, col=1)
fig.add_trace(text_trace_2, row=1, col=1)

# Set the camera to view along the z-axis (xy plane)
fig.update_layout(
    scene_camera=dict(
        eye=dict(x=2.5, y=0., z=0.)  # Adjust z to control the distance
    ),
    scene=dict(
        aspectmode='cube'  # Keeps the aspect ratio uniform
    )
)

############## add all the chase variables as line plots in second column ###############################
###### angle between the chasee and chaser ##############################################################
marker_trace_theta = go.Scatter(
    x=time_Axis[marker_indices],
    y=chase_df.loc[:,'chase_phi'][marker_indices],
    mode='markers',
    marker=dict(
        size=5,
        color='black',
        symbol='circle'
    ),
    name='phi'
)

fig.add_trace(go.Scatter(
    x = time_Axis, y = chase_df.loc[:,'chase_phi'],
    mode = 'lines'),
              row = 1, col = 3)
fig.add_trace(marker_trace_theta, row = 1, col = 3)


####### turning angle for the chaser ####################################################################
omega = angular_speed_df.loc[:,'chaser_phi']
omega_smoothed = signal.savgol_filter(omega, 10, 3)


fig.add_trace(go.Scatter(
        x = time_Axis, y = omega_smoothed
        , mode = 'lines'
        ),
             row = 2, col = 3)

######## rate of turning angle for the chaser ############################################################
omega_hat = np.diff(omega_smoothed)

marker_indices = np.arange(0, len(omega_hat), 20)
marker_trace_omega_hat = go.Scatter(
    x=time_Axis[marker_indices], 
    y=omega_hat[marker_indices],
    mode='markers',
    marker=dict(
        size=5,
        color='black',
        symbol='circle'
    ),
    name='omega_hat'
)

fig.add_trace(go.Scatter(
    x = time_Axis, y = omega_hat,
    mode = 'lines'),
              row = 3, col = 3)
fig.add_trace(marker_trace_omega_hat, row = 3, col = 3)

# Add manual annotations for column 3 titles
titles = [
    "Φ (angle between chaser and chasee)",
    "ω (turning angle of chaser)",
    "ω̂ (rate of turning angle of chaser)"
]

for i, title in enumerate(titles):
    fig.add_annotation(
        text=title,
        xref="paper",
        yref="paper",
        x=0.85,  # far right (col 3)
        y=1.0 - (i * 1 / 2.8),  # top, middle, bottom
        xanchor="center",
        yanchor="bottom",
        showarrow=False,
        font=dict(size=12, color="black")
    )

# Final layout tweaks
fig.update_layout(
    height=800,
    width=1000,
#     title_text="Custom Titles for Column 3 Subplots",
    showlegend=False
)

fig.update_layout({
    'xaxis3': dict(title='Time (miliseconds)'),
    "yaxis3": dict(title="ω̂ (degree/s² * 1e3)", showgrid=True)
    })


fig.show()
# fig.write_html("../dataFolders/MuscaChasingMusca/Sampled1K_Figures/Chase_relationships/" + name + "phi_visualization.html")
fig.write_html("../dataFolders/MuscaChasingMusca/Outputs/" + name + "phi_visualization.html")

In [32]:
cross_Cor = np.correlate(omega_hat, chase_df.loc[:,'chase_phi'], mode='full')
x_Axis = np.arange(-len(omega_hat), len(omega_hat), 1)
fig = go.Figure(data=go.Scatter(x = x_Axis, y=cross_Cor))
fig.show()
fig.write_html("../dataFolders/MuscaChasingMusca/Sampled1K_Figures/Chase_relationships/" + name + "phi_cross_cor.html")

In [ ]:
# fig = make_subplots(
#     rows=3, cols=2,
#     specs=[[{"type": "scene", "rowspan": 3}, {"type": "xy"}],
#            [None, {"type": "xy"}],
#            [None, {"type": "xy"}]
#            ])


# fig.add_trace(go.Scatter3d(
# x = com_df.chaser_com_x, y = com_df.chaser_com_y, z = com_df.chaser_com_z
#     ,mode = 'lines'
# , line=dict(color='black')), 
# row = 1, col = 1)

# # Set the camera to view along the z-axis (xy plane)
# fig.update_layout(
#     scene_camera=dict(
#         eye=dict(x=2.5, y=0, z=0)  # Adjust x to control the distance
#     ),
#     scene=dict(
#         aspectmode='cube'  # Keeps the aspect ratio uniform
#     )
# )


# fig.add_trace(go.Scatter3d(
# x = com_df.chasee_com_x, y = com_df.chasee_com_y, z = com_df.chasee_com_z
#     ,mode = 'lines'
#     , line=dict(color='red')), 
# row = 1, col = 1)

# #add all the line plots in second column
# # angle between the chasee and chaser
# fig.add_trace(go.Scatter(
#         x = time_Axis, y = chase_df.loc[:,'chase_phi']
#         , mode = 'lines'
#         ),
#              row = 1, col = 2)


# omega_hat = angular_speed_df.loc[:,'chaser_phi']
# omega_hat_smoothed = signal.savgol_filter(omega_hat, 10, 3)



# # turning angle for the chaser
# fig.add_trace(go.Scatter(
#         x = time_Axis, y = omega_hat_smoothed
#         , mode = 'lines'
#         ),
#              row = 2, col = 2)

# # rate of turning angle for the chaser
# fig.add_trace(go.Scatter(
#         x = time_Axis, y = np.diff(omega_hat_smoothed)
#         , mode = 'lines'
#         ),
#              row = 3, col = 2)
# fig.update_layout(
#     xaxis_title="Time (msecond)"
# )

In [ ]:
# fig.add_trace(go.Scatter3d(
# x = com_df.chaser_com_x, y = com_df.chaser_com_y, z = com_df.chaser_com_z
#     ,mode = 'lines'
# , line=dict(color='black')), 
# row = 1, col = 1)

# # Set the camera to view along the z-axis (xy plane)
# fig.update_layout(
#     scene_camera=dict(
#         eye=dict(x=0., y=0., z=2.5)  # Adjust z to control the distance
#     ),
#     scene=dict(
#         aspectmode='cube'  # Keeps the aspect ratio uniform
#     )
# )


# fig.add_trace(go.Scatter3d(
# x = com_df.chasee_com_x, y = com_df.chasee_com_y, z = com_df.chasee_com_z
#     ,mode = 'lines'
#     , line=dict(color='red')), 
# row = 1, col = 1)